In [18]:
import pandas as pd
import funzioni as fx
from tensorflow.keras.models import load_model
from tensorflow_addons.metrics import F1Score
import yfinance as yf

simbolo='IDT'

f1_score = F1Score(num_classes=1, average='macro', threshold=0.5)
model = load_model("model.h5", custom_objects={'Addons>F1Score': f1_score})
ticker = yf.download(simbolo, start='2010-01-01', end='2023-12-31', progress=False)

ticker.index = ticker.index.date
ticker = fx.crea_indicatori(ticker)
ticker = fx.imposta_target(ticker)
ticker.dropna(axis=0, inplace=True)
idx, X, Y, _ = fx.to_XY(ticker, fx.features_prezzo, fx.features_da_scalare_singolarmente, fx.features_meno_piu, fx.features_candele, fx.features_no_scala, fx.elenco_targets, fx.n_timesteps, fx.giorni_previsione, 0)
pred = model.predict(X)
pred = pred.reshape(-1,)
Y = Y.reshape(-1,)
df = pd.DataFrame({"Pred": pred, "Real": Y}, index=idx)
df.to_excel(f"previsioni/{simbolo}.xlsx")

102/102 [==============================] - 1s 4ms/step
